In [1]:
import os
import numpy as np
import pandas as pd


BASE = "/Users/jsh/Desktop/assignment2 spatial"

IN_CSV = os.path.join(BASE, "stage3_inputs_top20_fixed_dist.csv") 
OUT_DIR = os.path.join(BASE, "tables_stage3")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_ALL_CSV = os.path.join(BASE, "stage3_outputs_all20_coursework_baseline.csv")

OUT_TOP10_M_TEX = os.path.join(OUT_DIR, "table_top10_merchant.tex")
OUT_TOP10_S_TEX = os.path.join(OUT_DIR, "table_top10_supported.tex")
OUT_ALL20_TEX   = os.path.join(OUT_DIR, "table_all20_stage3.tex")
OUT_ASSUMP_TEX  = os.path.join(OUT_DIR, "table_stage3_assumptions.tex")


P_MW = 500
N_YEARS = 25

r_npv  = 0.05   
r_lcoe = 0.07   

# Electricity prices
p_merchant_gbp_per_MWh  = 80.0   
p_supported_gbp_per_MWh = 92.0   
c_cap_usd_per_MW = 2_000_000.0        
c_grid_usd_per_MW_km = 590.0          

USD_TO_GBP = 80.0 / 103.0

c_cap_gbp_per_MW = c_cap_usd_per_MW * USD_TO_GBP
c_grid_gbp_per_MW_km = c_grid_usd_per_MW_km * USD_TO_GBP

c_om_gbp_per_MW_yr = 0.0

AVAIL = 1.0


def crf(r, n):
    return (r * (1 + r) ** n) / ((1 + r) ** n - 1)

def npv_annuity(capex, revenue_yr, opex_yr, r, n):
    """Constant annual cashflow PV (annuity) approach."""
    annuity_pv = (revenue_yr - opex_yr) * (1 - (1 + r) ** (-n)) / r
    return -capex + annuity_pv

def save_latex_table(df_in, out_tex, cols, colnames, col_format, float_fmt="%.2f"):
    out = df_in[cols].copy()
    out.columns = colnames
    latex = out.to_latex(
        index=False,
        float_format=float_fmt,
        escape=False,
        column_format=col_format
    )
    with open(out_tex, "w") as f:
        f.write(latex)
    print("Wrote:", out_tex)


df = pd.read_csv(IN_CSV)

required = ["rank", "x", "y", "cf_fin", "d_tx_km", "d_sub_eff_km", "d_road_km"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in {IN_CSV}: {missing}")

HOURS = 8760.0
df["E_MWh_yr"] = P_MW * HOURS * df["cf_fin"] * AVAIL
df["R_m_gbp_yr"] = df["E_MWh_yr"] * p_merchant_gbp_per_MWh
df["R_s_gbp_yr"] = df["E_MWh_yr"] * p_supported_gbp_per_MWh

)
df["d_grid_km"] = np.minimum(df["d_sub_eff_km"], df["d_tx_km"])

df["CAPEX0_gbp"] = c_cap_gbp_per_MW * P_MW
df["CAPEX_grid_gbp"] = c_grid_gbp_per_MW_km * P_MW * df["d_grid_km"]
df["CAPEX_total_gbp"] = df["CAPEX0_gbp"] + df["CAPEX_grid_gbp"]

df["OPEX_gbp_yr"] = c_om_gbp_per_MW_yr * P_MW

CRF = crf(r_lcoe, N_YEARS)

df["LCOE_gbp_per_MWh"] = (df["CAPEX_total_gbp"] * CRF + df["OPEX_gbp_yr"]) / df["E_MWh_yr"]

df["NPV_m_gbp"] = npv_annuity(df["CAPEX_total_gbp"], df["R_m_gbp_yr"], df["OPEX_gbp_yr"], r_npv, N_YEARS)
df["NPV_s_gbp"] = npv_annuity(df["CAPEX_total_gbp"], df["R_s_gbp_yr"], df["OPEX_gbp_yr"], r_npv, N_YEARS)


df.to_csv(OUT_ALL_CSV, index=False)
print("Saved full outputs:", OUT_ALL_CSV)


print("\nLCOE summary:\n", df["LCOE_gbp_per_MWh"].describe())
print("\nNPV merchant summary:\n", df["NPV_m_gbp"].describe())
print("\nNPV supported summary:\n", df["NPV_s_gbp"].describe())

top10_m = df.sort_values("NPV_m_gbp", ascending=False).head(10).copy()
top10_s = df.sort_values("NPV_s_gbp", ascending=False).head(10).copy()

for _d in (df, top10_m, top10_s):
    _d["CAPEX_£bn"] = _d["CAPEX_total_gbp"] / 1e9
    _d["NPV_m_£bn"] = _d["NPV_m_gbp"] / 1e9
    _d["NPV_s_£bn"] = _d["NPV_s_gbp"] / 1e9


save_latex_table(
    top10_m,
    OUT_TOP10_M_TEX,
    cols=["rank","cf_fin","d_tx_km","d_sub_eff_km","d_grid_km","d_road_km","CAPEX_£bn","LCOE_gbp_per_MWh","NPV_m_£bn","NPV_s_£bn"],
    colnames=["rank","CF","d$_{tx}$(km)","d$_{sub,eff}$(km)","d$_{grid}$(km)","d$_{road}$(km)",
              "CAPEX(£bn)","LCOE(£/MWh)","NPV$_m$(£bn)","NPV$_s$(£bn)"],
    col_format="r r r r r r r r r r"
)

save_latex_table(
    top10_s,
    OUT_TOP10_S_TEX,
    cols=["rank","cf_fin","d_tx_km","d_sub_eff_km","d_grid_km","d_road_km","CAPEX_£bn","LCOE_gbp_per_MWh","NPV_m_£bn","NPV_s_£bn"],
    colnames=["rank","CF","d$_{tx}$(km)","d$_{sub,eff}$(km)","d$_{grid}$(km)","d$_{road}$(km)",
              "CAPEX(£bn)","LCOE(£/MWh)","NPV$_m$(£bn)","NPV$_s$(£bn)"],
    col_format="r r r r r r r r r r"
)


save_latex_table(
    df.sort_values("rank"),
    OUT_ALL20_TEX,
    cols=["rank","x","y","cf_fin","d_tx_km","d_sub_eff_km","d_grid_km","d_road_km","CAPEX_£bn","LCOE_gbp_per_MWh","NPV_m_£bn","NPV_s_£bn"],
    colnames=["rank","x","y","CF","d$_{tx}$(km)","d$_{sub,eff}$(km)","d$_{grid}$(km)","d$_{road}$(km)",
              "CAPEX(£bn)","LCOE(£/MWh)","NPV$_m$(£bn)","NPV$_s$(£bn)"],
    col_format="r r r r r r r r r r r r"
)

assump = pd.DataFrame([
    ["Installed capacity per site", "P (MW)", f"{P_MW:.0f}", "Coursework requirement (0.5 GW each)"],
    ["Project lifetime", "N (years)", f"{N_YEARS:d}", "Coursework example"],
    ["Discount rate (NPV)", "r (-)", f"{r_npv:.2f}", "Coursework example"],
    ["Discount rate (LCOE)", "r (-)", f"{r_lcoe:.2f}", "Within coursework range 5–8%"],
    ["Merchant electricity price", "p_m (GBP/MWh)", f"{p_merchant_gbp_per_MWh:.0f}", "Coursework: 103 USD/MWh ≈ 80 GBP/MWh"],
    ["Supported (CfD-like) price", "p_s (GBP/MWh)", f"{p_supported_gbp_per_MWh:.0f}", "Not provided; scenario parameter (cite)"],
    ["Onshore installation CAPEX", "c_cap (USD/MW)", f"{c_cap_usd_per_MW:,.0f}", "Coursework baseline"],
    ["Grid connection (inland)", "c_grid (USD/(MW·km))", f"{c_grid_usd_per_MW_km:,.0f}", "Coursework baseline"],
    ["USD→GBP conversion", "USD_TO_GBP (-)", f"{USD_TO_GBP:.3f}", "Derived from coursework price equivalence"],
    ["Fixed O&M", "c_om (GBP/(MW·yr))", f"{c_om_gbp_per_MW_yr:,.0f}", "Coursework example uses 0 (simplified)"],
    ["Availability", "A (-)", f"{AVAIL:.2f}", "Not specified; set to 1.0 for strict baseline consistency"],
], columns=["Parameter", "Symbol / unit", "Value", "Source / note"])

assump_tex = assump.to_latex(
    index=False,
    escape=False,
    column_format="p{4.2cm} p{3.2cm} p{2.6cm} p{5.6cm}"
)
with open(OUT_ASSUMP_TEX, "w") as f:
    f.write(assump_tex)
print("Wrote:", OUT_ASSUMP_TEX)

print("\nDone. LaTeX tables saved in:", OUT_DIR)


Saved full outputs: /Users/jsh/Desktop/assignment2 spatial/stage3_outputs_all20_coursework_baseline.csv

LCOE summary:
 count     20.000000
mean      92.826529
std       17.080458
min       47.017750
25%       88.542803
50%       97.832495
75%      104.855704
max      111.437065
Name: LCOE_gbp_per_MWh, dtype: float64

NPV merchant summary:
 count    2.000000e+01
mean     7.120375e+07
std      2.264523e+08
min     -1.023478e+08
25%     -6.000154e+07
50%     -8.586585e+06
75%      7.258478e+07
max      8.221104e+08
Name: NPV_m_gbp, dtype: float64

NPV supported summary:
 count    2.000000e+01
mean     1.984704e+08
std      2.604225e+08
min     -1.195098e+06
25%      4.758673e+07
50%      1.068451e+08
75%      2.000375e+08
max      1.062006e+09
Name: NPV_s_gbp, dtype: float64
Wrote: /Users/jsh/Desktop/assignment2 spatial/tables_stage3/table_top10_merchant.tex
Wrote: /Users/jsh/Desktop/assignment2 spatial/tables_stage3/table_top10_supported.tex
Wrote: /Users/jsh/Desktop/assignment2 spatial

In [2]:
import matplotlib.pyplot as plt

BASE = "/Users/jsh/Desktop/assignment2 spatial"
IN = os.path.join(BASE, "stage3_outputs_all20_coursework_baseline.csv")

FIG_DIR = os.path.join(BASE, "figures_stage3")
os.makedirs(FIG_DIR, exist_ok=True)

df = pd.read_csv(IN).sort_values("rank")


def savefig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=300, bbox_inches="tight")
    plt.close()
    print("Saved:", path)


plt.figure()
plt.hist(df["LCOE_gbp_per_MWh"].dropna(), bins=10)
plt.xlabel("LCOE (GBP/MWh)")
plt.ylabel("Count")
savefig(os.path.join(FIG_DIR, "fig_stage3_lcoe_hist.png"))

plt.figure(figsize=(10,4))
x = np.arange(len(df))
plt.bar(x - 0.2, df["NPV_m_gbp"]/1e9, width=0.4, label="Merchant NPV (£bn)")
plt.bar(x + 0.2, df["NPV_s_gbp"]/1e9, width=0.4, label="Supported NPV (£bn)")
plt.axhline(0, linewidth=1)
plt.xticks(x, df["rank"].astype(int), rotation=0)
plt.xlabel("Candidate rank (from Stage 2)")
plt.ylabel("NPV (£bn)")
plt.legend()
savefig(os.path.join(FIG_DIR, "fig_stage3_npv_bar.png"))

plt.figure()
plt.scatter(df["cf_fin"], df["NPV_m_gbp"]/1e9)
plt.xlabel("Capacity factor (CF)")
plt.ylabel("Merchant NPV (£bn)")
savefig(os.path.join(FIG_DIR, "fig_stage3_cf_vs_npv_merchant.png"))

plt.figure()
plt.scatter(df["cf_fin"], df["NPV_s_gbp"]/1e9)
plt.xlabel("Capacity factor (CF)")
plt.ylabel("Supported NPV (£bn)")
savefig(os.path.join(FIG_DIR, "fig_stage3_cf_vs_npv_supported.png"))


Saved: /Users/jsh/Desktop/assignment2 spatial/figures_stage3/fig_stage3_lcoe_hist.png
Saved: /Users/jsh/Desktop/assignment2 spatial/figures_stage3/fig_stage3_npv_bar.png
Saved: /Users/jsh/Desktop/assignment2 spatial/figures_stage3/fig_stage3_cf_vs_npv_merchant.png
Saved: /Users/jsh/Desktop/assignment2 spatial/figures_stage3/fig_stage3_cf_vs_npv_supported.png
